In [56]:
import sys
sys.path.append('/app/')  # noqa
sys.path.append('/app/loopa')  # noqa

from jina import Document, DocumentArray, Flow, Executor, requests
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import argparse
from loopa.executors.cache import SBERTEncoderCache
from loopa.executors.index import FaissIndexer, IndexMerger
from loopa.config import CUDA_IF_AVAILABLE
import torch
import hashlib

def my_hash(s): 
    return str(int(hashlib.md5(str(s).encode('utf-8')).hexdigest(), 16))

In [2]:
torch.multiprocessing.set_start_method('spawn', force=True) 

In [13]:
OUT_DIR = Path("/data/imgflip/scrap_language_image_pairs_20220209/")
OUT_TEMPLATE_TEXTS_CSV = OUT_DIR/"template_texts.csv"

In [14]:
JINA_SBERT_EMBEDDING_TEMPLATE_TEXT_COLLECTION = "00_sbert_all_mpnet_base_v2_imgflip_template_1400k_embeddings"
MONGO_EMBEDDING_DB_NAME = 'memlish_db'

In [36]:
def text_input_docs(templates_df):    
    for idx, row in templates_df.iterrows():
        template_name = row["template_img_name"]
        try:
            template_text = row["text"]
            document_id = f"{template_name}_{my_hash(template_text)}"
            doc = Document(id=document_id, text=template_text)# tags={'filename': str(template_img_path)})
        except Exception as e:
            print(document_id)
            print(template_text)
            print(str(e))
        
        yield doc 

In [16]:
templates_df = pd.read_csv(OUT_TEMPLATE_TEXTS_CSV)

In [43]:
templates_df = templates_df[~templates_df['text'].isna()]

In [57]:
templates_df = templates_df[~templates_df.duplicated(["text", "template_img_name"])]

In [17]:
templates_df.head()

,url,text,alt,views,upvotes,comments,meme_home_page,template_name,template_img_name
0,i.imgflip.com/61ov27.jpg,accepting that ur child made a good point; MOM...,Moms be like | accepting that ur child made a...,7208,106,7,/i/61ov27,Drake-Hotline-Bling,Drake-Hotline-Bling.jpg
1,i.imgflip.com/61soau.jpg,Ground children by sending them to bed early a...,Mexican meme | Ground children by sending the...,5271,65,5,/i/61soau,Drake-Hotline-Bling,Drake-Hotline-Bling.jpg
2,i.imgflip.com/6205qv.jpg,"image tagged in memes,drake hotline bling","Drake Hotline Bling | image tagged in memes,dr...",1265,6,0,/i/6205qv,Drake-Hotline-Bling,Drake-Hotline-Bling.jpg
3,i.imgflip.com/623hal.jpg,"""Let's eat, Grandma.""; ""Let's eat Grandma.""","certified hood classic | ""Let's eat, Grandma....",196,6,0,/i/623hal,Drake-Hotline-Bling,Drake-Hotline-Bling.jpg
4,i.imgflip.com/62353w.jpg,"Keep focusing on new relationships, being prod...",3 years of abstinence | Keep focusing on new ...,229,6,0,/i/62353w,Drake-Hotline-Bling,Drake-Hotline-Bling.jpg


In [47]:
embedder_params = {
        "device":CUDA_IF_AVAILABLE
}

cache_params = {
    "embedder_params": embedder_params,
    "db_name": MONGO_EMBEDDING_DB_NAME, 
    "collection_name": JINA_SBERT_EMBEDDING_TEMPLATE_TEXT_COLLECTION,
    "embedding_field_name":'emb',
    "megabatch_size":8192
}

In [48]:
flow_text_encode = Flow().add(uses=SBERTEncoderCache, name="SBERT_encoder", uses_with=cache_params)
flow_text_encode

In [ ]:
with flow_text_encode:
    flow_text_encode.post(on='', inputs=text_input_docs(templates_df), request_size=10_000, show_progress=True)

           Flow@15[I]:🎉 Flow is ready to use!                                             
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:39205
	🔒 Private network:	172.20.0.5:39205
	🌐 Public address:	35.224.116.253:39205
⠹ Working... ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:00:02 estimating... 

  0%|          | 0/2 [00:00<?, ?it/s]

⠙ Working... ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:00:26 estimating... 

 50%|█████     | 1/2 [00:24<00:24, 24.39s/it]

⠙ Working... ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:00:32 estimating... 

...

⠇ Working... ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━ 0:15:51  0.0 step/s 

100%|██████████| 2/2 [00:45<00:00, 22.72s/it]


⠋ Working... ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 0:15:57  0.0 step/s 

  0%|          | 0/2 [00:00<?, ?it/s]

⠧ Working... ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 0:16:33  0.0 step/s 